# convert ICON SCM output to DEPHY format

Code to read ICON SCM output files and write to DEPHY format (NetCDF)

Contributed by Martin Köhler from DWD (based on Ann. Fridlind's example)

## Import libraries

In [11]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import netCDF4
import datetime as dt
from netCDF4 import Dataset

In [12]:
### Functions for optical depth calculation

In [13]:
def intialise_sizeDistri():
    global a_CD,b_CD,nuCD,muCD,a_RD,b_RD,nuRD,muRD,a_ID,b_ID,nuID,muID,a_SD,b_SD,nuSD,muSD
    global a_GD,b_GD,nuGD,muGD,xxCD,xxRD,xxID,xxSD,xxGD,ddCD,ddRD,ddID,ddSD,ddGD

    #collection of parameters of size distributions
    #cloud= cloud_nue1mue1:
    a_CD = .124 #m/kg^beta      #D=ax^b
    b_CD = 1./3.
    nuCD = 1.                    #nu of f(x) cloud
    muCD = 1.                    #mu  "  " cloud

    #rain= rainSBB:
    a_RD = 0.124
    b_RD = 1./3.
    nuRD = 1.0                 
    muRD = 1./3.                  

    #ice= ice_cosmo5:
    a_ID = 0.835
    b_ID = 0.39
    nuID = 0.0              
    muID = 1./3. 

    #snow= snowSBB:
    a_SD = 5.13
    b_SD = 0.5
    nuSD = 0.0               
    muSD = 0.5

    #graupel= graupelhail_cosmo5:
    a_GD = 0.142
    b_GD = 0.314
    nuGD = 1.0                 
    muGD = 1./3. 

    #size ranges [x_min,x_max]
    xxCD = [4.2e-15, 2.6e-10] # kg
    xxRD = [2.6e-10, 6.5e-5] # kg
    xxID = [1.0e-12, 1.0e-5] # kg
    xxSD = [1.0e-10, 2.0e-5] # kg
    xxGD = [4.19e-09, 5.3e-4] # kg

    ddCD = a_CD*np.array(xxCD)**b_CD
    ddRD = a_RD*np.array(xxRD)**b_RD
    ddID = a_ID*np.array(xxID)**b_ID
    ddSD = a_SD*np.array(xxSD)**b_SD
    ddGD = a_GD*np.array(xxGD)**b_GD

def gamma_dist(x, A, nu, ll, mu):
    return A[None,:,:]*(x**nu)*np.exp(-ll[None,:,:]*(x**mu))

def transform_x2r(a_CD,b_CD,nuCD,muCD,qc,nc):
    xmCD = np.float64(qc)/np.float64(nc)
    dmCD = a_CD*xmCD**b_CD
    #lambda and A of size distribution (Seifert & Beheng 2006)
    lCD = (xmCD*gamma((nuCD+1)/muCD)/gamma((nuCD+2)/muCD))**(-muCD)
    Acd = muCD*nc*(lCD**((nuCD+1)/muCD))/gamma((nuCD+1)/muCD)
    #conversion constants to get to f(r)=K*r**psi*exp(-delta*r**xi)
    Kcd = (Acd/b_CD)*(2/a_CD)**((nuCD+1)/b_CD)
    dltCD = lCD*(2/a_CD)**(muCD/b_CD)
    psiCD = (nuCD+1)/b_CD-1
    xiCD = muCD/b_CD
    return Kcd,dltCD,psiCD,xiCD

def Ap_r(Dlim, K, psi, dlt, xi, n=1000):
    r_bounds = np.linspace(*Dlim/2,n)
    r_diff = np.diff(r_bounds)
    r_bin = (r_bounds[:-1] + r_bounds[1:]) / 2
    p = gamma_dist(r_bin[:, None, None], K, psi, dlt, xi)
    
    return np.sum(math.pi*r_bin[:, None, None]**2*p*r_diff[:, None, None], axis=0)

In [14]:
### Other functions for variable computations

In [15]:
#calculate vapour pressure
def calc_e(p,q):
    return q*p/(0.622 +(1-0.622)*q)

def calc_e_is(T):
     return np.exp(9.550426 - (5723.265 / T) + 3.53068 * np.log(T) - 0.00728332 * T)

def potential_temperature(temperature, pressure, reference_pressure=100000):
    """
    Calculate potential temperature from temperature and pressure.
    
    Args:
    - temperature: Temperature in Kelvin
    - pressure: Pressure in Pascals
    - reference_pressure: Reference pressure in Pascals (default: 100000 Pa)
    
    Returns:
    - Potential temperature in Kelvin
    """
    R = 287.05  # Specific gas constant for dry air (J/(kg·K))
    Cp = 1005  # Specific heat capacity at constant pressure for dry air (J/(kg·K))
    
    potential_temp = temperature * (reference_pressure / pressure) ** (R / Cp)
    
    return potential_temp

## Specify directory locations

If on the ARM JupyterHub, it is recommended to create and specify a local directory that is outside of the COMBLE-MIP repository to upload raw model output files in your model's format. 

Processed domain-mean outputs are invited for commit to the GitHub repository on a user-specified branch under /comble-mip/output_scm/YOUR_MODEL_NAME/sandbox/YOUR_OUTPUT_FILE_NAME. These can be committed and removed at any time.

If you are able to make a run without ice, it is requested to append 'noice' to YOUR_OUTPUT_FILE_NAME, so that it can readily be automatically compared with the baseline and other liquid-only runs.

In [16]:
# specify input and output file names: versions of ICON-SCM with and without ice

# Phys
my_input_suffix  = 'FixN_def_z0.nc'
my_output_suffix = 'FixN_def_z0.nc'

# Phys without ice
# my_input_suffix = 'entppe_noice.nc'
# my_output_suffix = 'Phys_FixN_noice.nc'

# Tun1
# my_input_suffix = 't698ml.nc'
# my_output_suffix = 'Tun1_FixN.nc'

# Tun1 without ice
# my_input_suffix = 't698ml_noice.nc'
# my_output_suffix = 'Tun1_FixN_noice.nc'

# Tun2
# my_input_suffix = 't705ml.nc'
# my_output_suffix = 'Tun2_FixN.nc'

# Tun2 without ice
# my_input_suffix = 't705ml_noice.nc'
# my_output_suffix = 'Tun2_FixN_noice.nc'

# specify local source directories (with subdirectories for spin up over ice and restart over water)
my_input_dir = '/user-data-home/comble-mip/output_scm/icon-scm/input/'

# specify Github scratch directory where processed model output will be committed (automate later)
my_output_filename = 'ICON-SCM_' + my_output_suffix
my_gitdir = '/user-data-home/comble-mip/output_scm/icon-scm/sandbox/'

## Read ICON-SCM data

### Read single file containing all output data

Note: ERROR 1: PROJ... message can be ignored here.

In [18]:
input_filename = my_input_dir + 'ICON-SCM_' + my_input_suffix
model_data = xr.open_dataset(input_filename)

# check if the run contains ice variables
do_ice = bool(max(model_data['tqi'].values)>0.)
print('do_ice = ',do_ice)

# data variable equality
icon_snds = model_data

# full parameter list
model_data

do_ice =  True


<xarray.Dataset>
Dimensions:           (time: 121, height: 90, bnds: 2, height_2: 91,
                       height_3: 91)
Coordinates:
  * time              (time) datetime64[ns] 2020-03-12T22:00:00 ... 2020-03-1...
    clon              float64 ...
    clat              float64 ...
  * height            (height) float64 1.0 2.0 3.0 4.0 ... 87.0 88.0 89.0 90.0
  * height_2          (height_2) float64 1.0 2.0 3.0 4.0 ... 88.0 89.0 90.0 91.0
  * height_3          (height_3) float64 1.0 2.0 3.0 4.0 ... 88.0 89.0 90.0 91.0
Dimensions without coordinates: bnds
Data variables: (12/52)
    height_bnds       (height, bnds) float64 ...
    height_2_bnds     (height_2, bnds) float64 ...
    gz0               (time) float32 ...
    pres_sfc          (time) float32 ...
    t_g               (time) float32 ...
    umfl_s            (time) float32 ...
    ...                ...
    ddt_temp_gscp     (time, height) float32 ...
    ddt_temp_turb     (time, height) float32 ...
    ddt_temp_pconv    (time, height) float32 ...
    ddt_qv_gscp       (time, height) float32 ...
    ddt_qv_turb       (time, height) float32 ...
    ddt_qv_conv       (time, height) float32 ...
Attributes:
    CDI:          Climate Data Interface version 2.3.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    uuidOfVGrid:  e0670a27-30fa-0503-fcfe-c68dbca12220
    source:       git@gitlab.dkrz.de:icon/icon-nwp.git@de4bad73b7366af2f51917...
    institution:  Max Planck Institute for Meteorology/Deutscher Wetterdienst
    title:        ICON simulation
    history:      Mon Mar 18 12:22:26 2024: ncwa -a ncells out_SCM_COMBLE_dep...
    references:   see MPIM/DWD publications
    comment:      unknown user name on vh1l102 (Linux 4.18.0-372.32.1.el8_6.x...
    CDO:          Climate Data Operators version 2.3.0 (https://mpimet.mpg.de...
    NCO:          netCDF Operators version 4.8.1 (Homepage = http://nco.sf.ne...

In [19]:
### Calculate and append additional variables

In [20]:
#get array dimension of [time,height] array
dims=icon_snds["rho"].data.shape
dims

#test

#surface friction velocity
#calculate using rho near surface
ustar=np.sqrt(np.sqrt(icon_snds["umfl_s"].data**2 + icon_snds["vmfl_s"].data**2)/icon_snds["rho"].data[:,dims[1]-1])
ustar[0]=0.000000001 # fix 0 for division in obukhov length calculation
icon_snds=icon_snds.assign(variables={'ustar': (('time'),ustar)})

#calculate surface roughness length
z0 = 9.0e-4 + 0.0*ustar
if 'def_z0' in my_output_filename:
    z0h = np.copy(z0)
else:
    z0h = 5.5e-6+0.0*np.copy(z0)

icon_snds=icon_snds.assign(variables={'z0':  (('time'),z0)})
icon_snds=icon_snds.assign(variables={'z0h': (('time'),z0h)})
icon_snds=icon_snds.assign(variables={'z0q': (('time'),z0h)})

#obukhov length
k = 0.4 #van Karman constant
g = 9.80665  #gravitational acceleration
rd= 287.04   #specific gas constant of dry air
cpd = 1004.64 # heat capcity of dry air at const. presssure
p0ref = 100000.0 # atm. reference pressure [Pa]
exner = np.exp(rd/cpd*np.log(icon_snds["pres_sfc"].data/p0ref))
theta_sfc = icon_snds["t_g"].data/exner

obk_length = - theta_sfc/ustar**3/(k*g*icon_snds["shfl_s"].data)
icon_snds=icon_snds.assign(variables={'ol': (('time'),obk_length)})

#atmosphere mass content of rain water
dz = (icon_snds['z_ifc'][0:-1].data-icon_snds['z_ifc'][1:].data)
rwp=np.sum(icon_snds['rho']*icon_snds['qr']*dz,axis=1)
icon_snds=icon_snds.assign(variables={'lwpr': (('time'),rwp.data)})

#atmosphere mass content of total ice water
iwp_tot=np.sum(icon_snds['rho']*(icon_snds['tot_qi_dia']+icon_snds['qs']+icon_snds['qg'])*dz,axis=1)
icon_snds=icon_snds.assign(variables={'iwp': (('time'),iwp_tot.data)})

#calculate ice-phase precipitation flux at surface
pri=icon_snds['ice_gsp_rate'].data + icon_snds['snow_gsp_rate'].data + icon_snds['graupel_gsp_rate'].data
icon_snds=icon_snds.assign(variables={'pri': (('time'),pri)})

#calculate relative humidity over ice
rhi=calc_e(icon_snds['pres'].data,icon_snds['qv'].data)/calc_e_is(icon_snds['temp'].data)
icon_snds=icon_snds.assign(variables={'huri': (('time','height'),rhi)})

#calculate potential temperature
theta=potential_temperature(icon_snds['temp'].data, icon_snds['pres'].data)
icon_snds=icon_snds.assign(variables={'theta': (('time','height'),theta)})

#calculate resolved TKE
#uprime=icon_snds["u"].data - icon_snds["u"].data.mean(axis=2)[:,:,None]
#vprime=icon_snds["v"].data - icon_snds["v"].data.mean(axis=2)[:,:,None]
#wprime=icon_snds["w"].data - icon_snds["w"].data.mean(axis=2)[:,:,None]
#tke_res=(0.5*np.sqrt(uprime**2 + vprime**2 + wprime**2)).mean(axis=2)
#wind_snds=wind_snds.assign(variables={'tke_res': (('time','height'),tke_res)})

#incloud droplet number concentration
#calculate nd fldmean only where qc>0.01g/kg
#nl_incloud=np.nanmean(np.where(icon_3D["qc"].data>1.0e-05,icon_3D["qnc"].data,np.nan),axis=2)
#set np.nan to "0"
#nl_incloud=np.where(np.isnan(nl_incloud),0,nl_incloud)
#icon_snds=icon_snds.assign(variables={'nlcic': (('time','height'),nl_incloud)})

#incloud total ice crystal number concentration
#computed only where qc>0.01g/kg & qi>1.0e-10
#ni_incloud=np.nanmean(np.where(np.logical_and(icon_3D["qc"].data>1.0e-05, 
#                               icon_3D["qi"].data+icon_3D["qs"].data+icon_3D["qg"].data > 1.0e-10),
#                               icon_3D["qni"].data+icon_3D["qns"].data+icon_3D["qng"].data,np.nan),axis=2)
#icon_snds=icon_snds.assign(variables={'niic': (('time','height'),ni_incloud)})

#calculate upward longwave flux
#icon_snds=icon_snds.assign(variables={'thd_s': (('time'), icon_snds["thu_s"].data - icon_snds["thb_s"].data)})


In [22]:
#### Prepare output file in DEPHY format

### Read requested variables list

Variable description, naming, units, and dimensions.

In [23]:
# read list of requested variables
vars_mean_list = pd.read_excel('https://docs.google.com/spreadsheets/d/1Vl8jYGviet7EtXZuQiitrx4NSkV1x27aJAhxxjBb9zI/export?gid=1026157027&format=xlsx',
                              sheet_name='SCM')
pd.set_option('display.max_rows', None)
vars_mean_list

,standard_name,variable_id,units,dimensions,"comment (reported at end of each model physics time step, green=minimum, red=granularity enabling EMC2)"
0,time,time,s,–,"dimension, seconds since 2020-03-12 18:00:00"
1,pressure_layer,layer,1,–,"dimension, pressure layer number from 1 at sur..."
2,air_pressure,pa,Pa,"time, layer",pressure at mid-level points (native model lev...
3,layer_top_pressure,pe,Pa,"time, layer","dimension, pressure at layer top points (used ..."
4,surface_pressure,ps,Pa,time,–
5,surface_temperature,ts,K,time,–
6,surface_friction_velocity,ustar,m s-1,time,–
7,surface_roughness_length_for_momentum_in_air,z0,m,time,–
8,surface_roughness_length_for_heat_in_air,z0h,m,time,–
9,surface_roughness_length_for_humidity_in_air,z0q,m,time,–


### Match ICON variables to requested outputs

Expand the table to include columns that indicate ICON model variable names and any conversion factor.

In [24]:
# drop comments
vars_mean_list = vars_mean_list.drop(columns='comment (reported at end of each model physics time step, green=minimum, red=granularity enabling EMC2)')

# add columns to contain model output name and units conversion factors
vars_mean_list = vars_mean_list.assign(model_name='missing data',conv_factor=1.0)

In [25]:
# match to ICON variable names and specify conversion factors
for index in vars_mean_list.index:
    standard_name = vars_mean_list.standard_name.iat[index]
    if standard_name=='surface_pressure': 
        vars_mean_list.model_name.iat[index] = 'pres_sfc'
        vars_mean_list.conv_factor.iat[index] = 100.
    if standard_name=='surface_temperature': 
        vars_mean_list.model_name.iat[index] = 't_g'
    if standard_name=='surface_friction_velocity': 
        vars_mean_list.model_name.iat[index] = 'ustar'
    if standard_name=='surface_roughness_length_for_momentum_in_air': 
        vars_mean_list.model_name.iat[index] = 'z0m'
    if standard_name=='surface_roughness_length_for_heat_in_air': 
        vars_mean_list.model_name.iat[index] = 'z0h'
    if standard_name=='surface_roughness_length_for_humidity_in_air': 
        vars_mean_list.model_name.iat[index] = 'z0q'
    if standard_name=='surface_upward_sensible_heat_flux': 
        vars_mean_list.model_name.iat[index] = 'shfl_s'
        vars_mean_list.conv_factor.iat[index] = -1.
    if standard_name=='surface_upward_latent_heat_flux': 
        vars_mean_list.model_name.iat[index] = 'lhfl_s'
        vars_mean_list.conv_factor.iat[index] = -1.
    if standard_name=='obukhov_length': 
        vars_mean_list.model_name.iat[index] = 'ol'
    if standard_name=='pbl_height': 
        vars_mean_list.model_name.iat[index] = 'pblht_bp'
    if standard_name=='inversion_height': 
        vars_mean_list.model_name.iat[index] = 'pblht_th'
    if standard_name=='atmosphere_mass_content_of_liquid_cloud_water': 
        vars_mean_list.model_name.iat[index] = 'clwpt'
        vars_mean_list.conv_factor.iat[index] = 0.001
    if standard_name=='atmosphere_mass_content_of_rain_water': 
        vars_mean_list.model_name.iat[index] = 'rlwpt'
        vars_mean_list.conv_factor.iat[index] = 0.001
    if do_ice:
        if standard_name=='atmosphere_mass_content_of_ice_water': 
            vars_mean_list.model_name.iat[index] = 'iwpt'
            vars_mean_list.conv_factor.iat[index] = 0.001
    if standard_name=='area_fraction_cover_of_hydrometeors': 
        vars_mean_list.model_name.iat[index] = 'clct'
#    if standard_name=='area_fraction_cover_of_liquid_cloud': 
#        vars_mean_list.model_name.iat[index] = ''
    if standard_name=='area_fraction_cover_of_convective_hydrometeors': 
        vars_mean_list.model_name.iat[index] = 'clct'
    if standard_name=='optical_depth': 
        vars_mean_list.model_name.iat[index] = 'tau'
#    if standard_name=='optical_depth_of_liquid_water': 
#        vars_mean_list.model_name.iat[index] = ''
    if standard_name=='precipitation_flux_at_surface': 
        vars_mean_list.model_name.iat[index] = 'tot_prec_rate'
        vars_mean_list.conv_factor.iat[index] = 1./86400
    if standard_name=='precipitation_flux_of_ice_at_surface': 
        vars_mean_list.model_name.iat[index] = 'snowfall'
        vars_mean_list.conv_factor.iat[index] = 1./86400
    if standard_name=='toa_outgoing_longwave_flux': 
        vars_mean_list.model_name.iat[index] = 'thb_t'
    if standard_name=='surface_downwelling_longwave_flux': 
        vars_mean_list.model_name.iat[index] = 'thd_s'  
    if standard_name=='surface_upwelling_longwave_flux': 
        vars_mean_list.model_name.iat[index] = 'thu_s'  
    if standard_name=='height': 
        vars_mean_list.model_name.iat[index] = 'z_mc'
    if standard_name=='eastward_wind': 
        vars_mean_list.model_name.iat[index] = 'u'
    if standard_name=='northward_wind': 
        vars_mean_list.model_name.iat[index] = 'v'
    if standard_name=='air_dry_density': 
        vars_mean_list.model_name.iat[index] = 'rho'
    if standard_name=='air_temperature': 
        vars_mean_list.model_name.iat[index] = 'temp'
    if standard_name=='water_vapor_mixing_ratio': 
        vars_mean_list.model_name.iat[index] = 'qv'
    if standard_name=='relative_humidity': 
        vars_mean_list.model_name.iat[index] = 'rh'
        vars_mean_list.conv_factor.iat[index] = 0.01
    if standard_name=='relative_humidity_over_ice': 
        vars_mean_list.model_name.iat[index] = 'rhi'
        vars_mean_list.conv_factor.iat[index] = 0.01
    if standard_name=='air_potential_temperature': 
        vars_mean_list.model_name.iat[index] = 'theta'
    if standard_name=='mass_mixing_ratio_of_cloud_liquid_water_in_air': 
        vars_mean_list.model_name.iat[index] = 'tot_qc_dia'
    if standard_name=='mass_mixing_ratio_of_rain_water_in_air': 
        vars_mean_list.model_name.iat[index] = 'qr'
    if do_ice: 
        if standard_name=='mass_mixing_ratio_of_ice_water_in_air': 
            vars_mean_list.model_name.iat[index] = 'qi'
    if standard_name=='area_fraction_of_hydrometeors': 
        vars_mean_list.model_name.iat[index] = 'clc'
    if standard_name=='area_fraction_of_liquid_cloud': 
        vars_mean_list.model_name.iat[index] = 'lcf'
    if standard_name=='area_fraction_of_convective_hydrometeors': 
        vars_mean_list.model_name.iat[index] = 'clc'
    if standard_name=='precipitation_flux_in_air': 
        vars_mean_list.model_name.iat[index] = 'prt'
        vars_mean_list.conv_factor.iat[index] = 1./86400
    if do_ice:
        if standard_name=='precipitation_flux_in_air_in_ice_phase': 
            vars_mean_list.model_name.iat[index] = 'pit'
            vars_mean_list.conv_factor.iat[index] = 1./86400
    if standard_name=='specific_turbulent_kinetic_energy': 
        vars_mean_list.model_name.iat[index] = 'tke'
    if standard_name=='disspation_rate_of_turbulent_kinetic_energy': 
        vars_mean_list.model_name.iat[index] = 'dissip_tke_turb'
        vars_mean_list.conv_factor.iat[index] = -1.
    if standard_name=='zonal_momentum_flux': 
        vars_mean_list.model_name.iat[index] = 'uw_turb'
    if standard_name=='meridional_momentum_flux': 
        vars_mean_list.model_name.iat[index] = 'vw_turb'
    if standard_name=='variance_of_upward_air_velocity': 
        vars_mean_list.model_name.iat[index] = 'w2_turb'
    if standard_name=='vertical_flux_potential_temperature': 
        vars_mean_list.model_name.iat[index] = 'wth_turb'
#    if standard_name=='vertical_flux_liquid_water_potential_temperature': 
#        vars_mean_list.model_name.iat[index] = ''
    if standard_name=='vertical_flux_water_vapor': 
        vars_mean_list.model_name.iat[index] = 'wq_turb'
    if standard_name=='vertical_flux_total_water': 
        vars_mean_list.model_name.iat[index] = 'wqt_turb'
    if standard_name=='convection_updraft_mass_flux': 
        vars_mean_list.model_name.iat[index] = 'lwdp'
    if standard_name=='convection_downdraft_mass_flux': 
        vars_mean_list.model_name.iat[index] = 'lwdp'
    if standard_name=='downwelling_longwave_flux_in_air': 
        vars_mean_list.model_name.iat[index] = 'lwdp'
    if standard_name=='upwelling_longwave_flux_in_air': 
        vars_mean_list.model_name.iat[index] = 'lwup'
    if standard_name=='tendency_of_air_potential_temperature_due_to_radiative_heating': 
        vars_mean_list.model_name.iat[index] = 'ddt_temp_radlw'
        vars_mean_list.conv_factor.iat[index] = 1./86400
    if standard_name=='tendency_of_air_potential_temperature_due_to_microphysics': 
        vars_mean_list.model_name.iat[index] = 'ddt_temp_gscp'
        vars_mean_list.conv_factor.iat[index] = 1./86400
    if standard_name=='tendency_of_air_potential_temperature_due_to_mixing': 
        vars_mean_list.model_name.iat[index] = 'ddt_temp_turb'
        vars_mean_list.conv_factor.iat[index] = 1./86400
    if standard_name=='tendency_of_water_vapor_mixing_ratio_due_to_microphysics': 
        vars_mean_list.model_name.iat[index] = 'ddt_qv_gscp'
        vars_mean_list.conv_factor.iat[index] = 1./86400
    if standard_name=='tendency_of_water_vapor_mixing_ratio_due_to_mixing': 
        vars_mean_list.model_name.iat[index] = 'ddt_qv_turb'
        vars_mean_list.conv_factor.iat[index] = 1./86400
    if standard_name=='mass_mixing_ratio_of_liquid_cloud_water_in_air_stratiform': 
        vars_mean_list.model_name.iat[index] = 'qcl'
    if standard_name=='mass_mixing_ratio_of_rain_water_in_air_stratiform': 
        vars_mean_list.model_name.iat[index] = 'qpl'
    if do_ice:
        if standard_name=='mass_mixing_ratio_of_ice_cloud_in_air_stratiform': 
            vars_mean_list.model_name.iat[index] = 'qci'
        if standard_name=='mass_mixing_ratio_of_ice_precipitation_in_air_stratiform': 
            vars_mean_list.model_name.iat[index] = 'qpi'
    if standard_name=='mass_mixing_ratio_of_liquid_cloud_water_in_air_convective': 
        vars_mean_list.model_name.iat[index] = 'QCLmc'
    if standard_name=='mass_mixing_ratio_of_rain_water_in_air_convective': 
        vars_mean_list.model_name.iat[index] = 'QPLmc'
    if do_ice:
        if standard_name=='mass_mixing_ratio_of_ice_cloud_in_air_convective': 
            vars_mean_list.model_name.iat[index] = 'QCImc'
        if standard_name=='mass_mixing_ratio_of_ice_precipitation_in_air_convective': 
            vars_mean_list.model_name.iat[index] = 'QPImc'
    if standard_name=='number_of_liquid_cloud_droplets_in_air_stratiform': 
        vars_mean_list.model_name.iat[index] = 'ncl'
    if standard_name=='number_of_rain_drops_in_air_stratiform': 
        vars_mean_list.model_name.iat[index] = 'npl'
    if do_ice:
        if standard_name=='number_of_ice_cloud_crystals_in_air_stratiform': 
            vars_mean_list.model_name.iat[index] = 'nci'
        if standard_name=='number_of_ice_precipitation_crystals_in_air_stratiform': 
            vars_mean_list.model_name.iat[index] = 'npi'
    if standard_name=='effective_radius_of_liquid_cloud_droplets_convective': 
        vars_mean_list.model_name.iat[index] = 're_mccl'
    if standard_name=='effective_radius_of_rain_convective': 
        vars_mean_list.model_name.iat[index] = 're_mcpl'
    if do_ice:
        if standard_name=='effective_radius_of_ice_cloud_convective': 
            vars_mean_list.model_name.iat[index] = 're_mcci'
        if standard_name=='effective_radius_of_ice_precipitation_convective': 
            vars_mean_list.model_name.iat[index] = 're_mcpi'
    if standard_name=='area_fraction_of_liquid_cloud_stratiform': 
        vars_mean_list.model_name.iat[index] = 'cldsscl'
    if standard_name=='area_fraction_of_rain_stratiform': 
        vars_mean_list.model_name.iat[index] = 'cldsspl'
    if do_ice:
        if standard_name=='area_fraction_of_ice_cloud_stratiform': 
            vars_mean_list.model_name.iat[index] = 'cldssci'
        if standard_name=='area_fraction_of_ice_precipitation_stratiform': 
            vars_mean_list.model_name.iat[index] = 'cldsspi'
    if standard_name=='area_fraction_of_liquid_cloud_convective': 
        vars_mean_list.model_name.iat[index] = 'cldmccl'
    if standard_name=='area_fraction_of_rain_convective': 
        vars_mean_list.model_name.iat[index] = 'cldmcpl'
    if do_ice:
        if standard_name=='area_fraction_of_ice_cloud_convective': 
            vars_mean_list.model_name.iat[index] = 'cldmcci'
        if standard_name=='area_fraction_of_ice_precipitation_convective': 
            vars_mean_list.model_name.iat[index] = 'cldmcpi'
    if standard_name=='mass_weighted_fall_speed_of_liquid_cloud_water_stratiform': 
        vars_mean_list.model_name.iat[index] = 'vm_sscl'
    if standard_name=='mass_weighted_fall_speed_of_rain_stratiform': 
        vars_mean_list.model_name.iat[index] = 'vm_sspl'
    if do_ice:
        if standard_name=='mass_weighted_fall_speed_of_ice_cloud_stratiform': 
            vars_mean_list.model_name.iat[index] = 'vm_ssci'
        if standard_name=='mass_weighted_fall_speed_of_ice_precipitation_stratiform': 
            vars_mean_list.model_name.iat[index] = 'vm_sspi'
    if standard_name=='mass_weighted_fall_speed_of_liquid_cloud_water_convective': 
        vars_mean_list.model_name.iat[index] = 'vm_mccl'
    if standard_name=='mass_weighted_fall_speed_of_rain_convective': 
        vars_mean_list.model_name.iat[index] = 'vm_mcpl'
    if do_ice:
        if standard_name=='mass_weighted_fall_speed_of_cloud_ice_crystals_convective': 
            vars_mean_list.model_name.iat[index] = 'vm_mcci'
        if standard_name=='mass_weighted_fall_speed_of_ice_precipitation_convective': 
            vars_mean_list.model_name.iat[index] = 'vm_mcpi'

vars_mean_list[2:] # echo variables (first two rows are dimensions)

,standard_name,variable_id,units,dimensions,model_name,conv_factor
2,air_pressure,pa,Pa,"time, layer",missing data,1.000000
3,layer_top_pressure,pe,Pa,"time, layer",missing data,1.000000
4,surface_pressure,ps,Pa,time,pres_sfc,100.000000
5,surface_temperature,ts,K,time,t_g,1.000000
6,surface_friction_velocity,ustar,m s-1,time,ustar,1.000000
7,surface_roughness_length_for_momentum_in_air,z0,m,time,z0m,1.000000
8,surface_roughness_length_for_heat_in_air,z0h,m,time,z0h,1.000000
9,surface_roughness_length_for_humidity_in_air,z0q,m,time,z0q,1.000000
10,surface_upward_sensible_heat_flux,hfss,W m-2,time,shfl_s,-1.000000
11,surface_upward_latent_heat_flux,hfls,W m-2,time,lhfl_s,-1.000000


## Create DEPHY output file

Write a single file to contain all domain-mean scalar and profile outputs. This code expects the write directory to be pre-existing (already created by the user). In the case that this output will be committed to the comble-mip GitHub repository, see above "Specify directory locations".

In [26]:
#print(model_data)
variable_names = list(model_data.data_vars.keys())
print(variable_names)

['height_bnds', 'height_2_bnds', 'gz0', 'pres_sfc', 't_g', 'umfl_s', 'vmfl_s', 'shfl_s', 'lhfl_s', 'tcm', 'tch', 'tqv', 'tqc', 'tqi', 'tqr', 'tqs', 'tqg', 'tot_prec_rate', 'ice_gsp_rate', 'snow_gsp_rate', 'graupel_gsp_rate', 'thu_s', 'thb_s', 'thb_t', 'sob_s', 'tqv_dia', 'tqc_dia', 'tqi_dia', 'z_mc', 'z_ifc', 'pres', 'u', 'v', 'rho', 'temp', 'tke', 'exner', 'qv', 'tot_qc_dia', 'tot_qi_dia', 'qr', 'qs', 'qg', 'clc', 'rh', 'ddt_temp_radlw', 'ddt_temp_gscp', 'ddt_temp_turb', 'ddt_temp_pconv', 'ddt_qv_gscp', 'ddt_qv_turb', 'ddt_qv_conv']


In [27]:
# create DEPHY output file
dephy_filename = my_gitdir + my_output_filename
if os.path.exists(dephy_filename):
    os.remove(dephy_filename)
    print('The file ' + dephy_filename + ' has been deleted successfully')    
dephy_file = Dataset(dephy_filename,mode='w',format='NETCDF3_CLASSIC')
start_date = '2020-03-12T22:00:00Z'

# create global attributes
dephy_file.title='ICON SCM results for COMBLE-MIP case: fixed stratiform Nd and Ni'
dephy_file.reference='https://github.com/ARM-Development/comble-mip'
dephy_file.authors='Martin Köhler (martin.koehler@dwd.de), Anna Possner (apossner@iau.uni-frankfurt.de)'
dephy_file.source=input_filename
dephy_file.version=dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
dephy_file.format_version='DEPHY SCM format version 1.6'
dephy_file.script='convert_ModelE3_SCM_output_to_dephy_format.ipynb'
dephy_file.startDate=start_date
dephy_file.force_geo=1
dephy_file.surfaceType='ocean'
dephy_file.surfaceForcing='ts'
dephy_file.lat='74.5 deg N'
dephy_file.dp='see pressure variable'
dims=icon_snds["pres"].data.shape
dephy_file.np=dims[1]

# create dimensions
nt = model_data.dims['time']
time = dephy_file.createDimension('time', nt)
time = dephy_file.createVariable('time', np.float64, ('time',))
time.units = 'seconds since ' + dephy_file.startDate
time.long_name = 'time'
# find time step and build time in seconds
time_str1 = str(model_data['time'].data[0]).split('.')[0]  # Removes fractional seconds
time_str2 = str(model_data['time'].data[1]).split('.')[0]  # Removes fractional seconds
time1 = dt.datetime.strptime(time_str1,'%Y-%m-%dT%H:%M:%S')
time2 = dt.datetime.strptime(time_str2,'%Y-%m-%dT%H:%M:%S')
delta_t = (time2-time1).total_seconds()
time[:] = (np.arange(nt)+1.)*delta_t

# vertical dimension: model layers called 'pressure_layer'
nl = dims[1]      # Number of levels
pressure_layer           = dephy_file.createDimension('pressure_layer', nl)
pressure_layer           = dephy_file.createVariable ('pressure_layer', np.int32, ('pressure_layer',))
pressure_layer.units     = 'unitless'
pressure_layer.long_name = 'pressure layer'
pressure_layer[:]        = np.arange(1, nl + 1)   # values from 1 to 90, 1 at the surface and 90 at the top


# Old fixed pressure layers
#nl = model_data.dims['pres']
#pa = dephy_file.createDimension('pa', nl)
#pa = dephy_file.createVariable('pa', np.float64, ('pa',))
#pa.units = 'Pa'
#pa.long_name = 'pressure'
#pa[:] = model_data['pres'].data*100.

# create and fill variables
for index in vars_mean_list.index[2:]:
    std_name = vars_mean_list.standard_name.iat[index]
#   print(std_name) # debug
    var_name = vars_mean_list.variable_id.iat[index]
    mod_name = vars_mean_list.model_name.iat[index]
    c_factor = vars_mean_list.conv_factor.iat[index]
    if vars_mean_list.dimensions.iat[index]=='time':
        new_sca           = dephy_file.createVariable(var_name, np.float64, ('time'))
        new_sca.units     = vars_mean_list.units.iat[index]
        new_sca.long_name = std_name
        print('variable:', mod_name, model_data[mod_name].data)
        if vars_mean_list.model_name.iat[index]!='missing data':
            new_sca[:]    = model_data[mod_name].data*c_factor
    if vars_mean_list.dimensions.iat[index]=='time, pressure_layer':
        new_snd           = dephy_file.createVariable(var_name, np.float64, ('time','pressure_layer'))
        new_snd.units     = vars_mean_list.units.iat[index]
        new_snd.long_name = std_name
        if vars_mean_list.model_name.iat[index]!='missing data': 
            new_snd[:]    = model_data[mod_name].data*c_factor

print(dephy_file)
dephy_file.close()

The file /user-data-home/comble-mip/output_scm/icon-scm/sandbox/ICON-SCM_FixN_def_z0.nc has been deleted successfully
variable: pres_sfc [ 99544.734  99503.164  99488.76   99495.68   99502.53   99505.36
  99506.03   99510.69   99527.77   99554.375  99587.62   99626.3
  99669.63   99745.41   99885.7   100079.13  100308.64  100586.34
 100889.07  101223.63  101565.71  101894.055 102225.56  102552.82
 102871.77  103176.14  103468.62  103735.56  103978.71  104215.57
 104441.97  104650.06  104856.19  105065.03  105264.7   105348.31
 105504.98  105589.44  105693.62  105805.98  105904.81  105996.38
 106088.47  106217.29  106295.39  106371.34  106448.25  106521.93
 106595.97  106660.23  106721.03  106788.85  106847.28  106963.11
 107011.41  107057.74  107168.99  107201.29  107271.984 107315.72
 107419.22  107425.586 107522.38  107542.13  107638.336 107645.35
 107739.9   107755.94  107813.5   107839.82  107874.23  107962.516
 107968.02  108054.76  108073.21  108096.31  108180.41  108184.03
 1082

KeyError: 'ustar'

### Check output file

In [9]:
dephy_check = xr.open_dataset(dephy_filename)
dephy_check

<xarray.Dataset>
Dimensions:    (time: 40, pa: 110)
Coordinates:
  * time       (time) datetime64[ns] 2020-03-12T22:30:00 ... 2020-03-13T18:00:00
  * pa         (pa) float64 9.79e+04 9.69e+04 9.59e+04 ... 1.4 0.75 0.35
Data variables: (12/94)
    ps         (time) float64 ...
    ts         (time) float64 ...
    ustar      (time) float64 ...
    z0         (time) float64 ...
    z0h        (time) float64 ...
    z0q        (time) float64 ...
    ...         ...
    vmics      (time, pa) float64 ...
    vmips      (time, pa) float64 ...
    vmlcc      (time, pa) float64 ...
    vmlrc      (time, pa) float64 ...
    vmicc      (time, pa) float64 ...
    vmipc      (time, pa) float64 ...
Attributes: (12/14)
    title:           ModelE3 SCM results for COMBLE-MIP case: fixed stratifor...
    reference:       https://github.com/ARM-Development/comble-mip
    authors:         Ann Fridlind (ann.fridlind@nasa.gov), Florian Tornow (fl...
    source:          /data/home/fridlind/modelE3//allsteps.allmergeSCM_COMBLE...
    version:         2023-11-30 00:16:50
    format_version:  DEPHY SCM format version 1.6
    ...              ...
    force_geo:       1
    surfaceType:     ocean
    surfaceForcing:  ts
    lat:             74.5 deg N
    dp:              see pressure variable
    np:              110